### Import used modules

In [1]:
%config Completer.use_jedi = False
### reload magic
%reload_ext autoreload
%autoreload 2
path, = !pwd
path = path.split('/')[:-1]
import os, sys, argparse
import torch
import numpy as np
from tqdm import tqdm
from datetime import datetime
from torch.utils.data import DataLoader
os.environ['ROOT_DATA_DIR']='/work/xc3vancechen/data/DLRA_database'
# utils和fast_eval.py不在同一層，所以要sys.path.extend
sys.path.extend(["/".join(path) + '/training'])

from utils.run_utils import (parse_date_end, parse_date_start,
                             get_model, checkpoint_parser)
from core.model_type import ModelType
from core.enum import DataType
from data_loaders.data_loader_all_loaded import DataLoaderAllLoaded
from utility import performance, idxFromDatetime
from plottingFunction import plotFig

os.environ["CUDA_VISIBLE_DEVICES"]="0" #only 1 GPU後面nn.parellel才不會出事-

/home/xc3vancechen/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### customized parameters

In [2]:
parser = argparse.ArgumentParser() #--代表是optional
parser.add_argument('--test_start', type=parse_date_start, default=datetime(2021, 12, 31))
parser.add_argument('--test_end', type=parse_date_end, default=datetime(2022, 12, 31, 23, 10))
parser.add_argument('--ckpt_dir', type=str, 
                    default="/".join(path) + '/training/checkpoints/')
args = parser.parse_args([]) # pulse on the parsers
checkpoint = sorted(os.listdir(args.ckpt_dir))
checkpoint = [args.ckpt_dir+x for x in checkpoint if 'mt-18_dt-81' in x][0]
print('model ckpt name: ',checkpoint)
s = args.test_start
e = args.test_end
input_shape=(540,420)
sampling_rate = 6 # examination per hour
is_test = True
batch_size = 32 # not necessary to be the same as the one training uesd # take "16" while evaluating LCL-2
num_workers = 8 # for Dataloader

model ckpt name:  /work/xc3vancechen/DLRA/training/checkpoints/RF_10101_41231_mt-18_dt-81_lt-0_tlen-3_ilen-6_sampl-6_teaf-0.5_AdvW-0.01_DisD-3_v-30_epoch=3_val_loss=1.526435_pdsr=0.33_D_auc=1.00_D_pos_acc=0.37_D_neg_acc=0.64.ckpt


### data args, model args

In [3]:
idx_lcl = 0
kwargs = checkpoint_parser(checkpoint)
for k in kwargs.keys():
    if k in ['lw','adv_w','AdvW','auc','D_pos_acc','D_neg_acc','D_auc','lssim','lmae', 'teaf']:
        kwargs[k] = float(kwargs[k])
    elif k not in ['start','end','val_loss','pdsr']:
        kwargs[k] = int(kwargs[k])
    
    if k in ['lcl',]:
        idx_lcl = int(kwargs[k])

        
if 'dt' not in kwargs:
    if bool(kwargs.get('jst_rain',0)):
        data_type = DataType.RAIN
    elif bool(kwargs.get('jst_radar',0)):
        data_type = DataType.RADAR
    else:
        data_type = DataType.RAIN + DataType.RADAR + DataType.ELEVATION
else:
    data_type=int(kwargs['dt'])

data_kwargs = {
    'data_type': data_type,
    'residual': bool(kwargs.get('res',0)),
    'target_offset': int(kwargs.get('toff', 0)),
    'target_len': int(kwargs['tlen']),
    'input_len': int(kwargs.get('ilen',6)),
    'hourly_data': bool(kwargs.get('hrly',0)),
    'hetero_data': bool(kwargs.get('hetr',0)),
    'sampling_rate': sampling_rate,
    'prior_dtype': DataType.NONEATALL,
    'random_std': int(kwargs.get('r_std', 0)),
    'threshold': 0.5,
}
model_kwargs = {
        'adv_w': float(kwargs.get('adv_w', 0.1)),
        'model_type': int(kwargs['mt']),
        # For adding locallty connected layers
        'LCL':int(kwargs.get('LCL', idx_lcl)),
        'dis_d': int(kwargs.get('dis_d', 3)),
        'teach_force': int(-1) # for testing
    }

loss_kwargs = {'type': kwargs['lt'], 
               'aggregation_mode': kwargs.get('la'), 
               'kernel_size': kwargs.get('lsz'),
               'w': float(kwargs.get('lw', 1)),
               'residual_loss':None,
               'mae_w':0.1,
               'ssim_w':0.02,
              }

In [4]:
print(idx_lcl)

0


### Load data & model, then evaluate

In [5]:

xxx_radar = np.loadtxt('/work/xc3vancechen/data/DLRA_database/radar_2d_compressed/2022/202207/20220701/20220701_0010.nc.gz', dtype=np.float32)
print("radar", np.max(xxx_radar[-1]))
# xxx_rain = np.loadtxt('/work/xc3vancechen/data/DLRA_database/rain_compressed_10minFinal/2022/202210/20221028/20221028_0610.nc.gz', dtype=np.float32)
# print("rain", xxx_rain)

radar 63.0


In [6]:
dataset = DataLoaderAllLoaded(s,e,
                              data_kwargs['input_len'],
                              data_kwargs['target_len'], 
                              workers=0,                               
                              target_offset=int(kwargs.get('toff',0)),
                              data_type=data_type,
                              is_train=True,
                              is_validation = False,
                              is_test = False,
                              img_size=input_shape,
                              residual=data_kwargs['residual'],
                              hourly_data=data_kwargs['hourly_data'],
                              hetero_data=data_kwargs['hetero_data'],
                              sampling_rate=sampling_rate,
                              threshold=data_kwargs['threshold']
                             )
loader = DataLoader(dataset, batch_size=batch_size,num_workers=num_workers,shuffle=False)

model = get_model(
        s,
        e,
        model_kwargs,
        loss_kwargs,
        data_kwargs,
        '',
        '',
        data_loader_info=dataset.get_info_for_model(),
        )

# GPU
checkpoint = torch.load(checkpoint)
_ = model.load_state_dict(checkpoint['state_dict'], strict=False)
model = torch.nn.DataParallel(model).cuda()

# CPU
# checkpoint = torch.load(checkpoint, map_location=torch.device('cpu'))
# _ = model.load_state_dict(checkpoint['state_dict'])

[Loading Train Data] 2021-12-31 00:00:00 2022-12-31 23:10:00


100%|██████████| 13/13 [00:21<00:00,  1.68s/it]


Latitude of Map: 21.9~25.3,Longitude of Map: 120.0~122.0


100%|██████████| 43953/43953 [00:01<00:00, 38560.99it/s]


Latitude Range: 20.125~26.8625, Longitude Range: 118.125~123.3625 for cropped TW [540x420]
Latitude of Map: 21.9~25.3,Longitude of Map: 120.0~122.0


100%|██████████| 43953/43953 [00:01<00:00, 38588.13it/s]


Latitude Range: 20.125~26.8625, Longitude Range: 118.125~123.3625 for cropped TW [540x420]
Latitude of Map: 21.9~25.3,Longitude of Map: 120.0~122.0


100%|██████████| 43953/43953 [00:01<00:00, 38653.37it/s]


Latitude Range: 20.125~26.8625, Longitude Range: 118.125~123.3625 for cropped TW [540x420]
First and last time steps: 2021-12-31 00:10:00 2022-12-31 23:10:00
Random time steps: 2022-05-04 23:10:00 2022-02-05 22:10:00 2022-05-12 10:30:00
[DataLoaderAllLoaded] Size:52675 Skipped:0
[DataLoaderAllLoaded Dtype] RAIN
[DataLoaderAllLoaded Dtype] RADAR
[DataLoaderAllLoaded Dtype] WIND
[DataLoaderAllLoaded Dtype] RAIN_RADAR
[DataLoaderAllLoaded Dtype] RAIN_RADAR_WIND
[DataLoaderAllLoaded] 2021-12-31 00:00:00<->2022-12-31 23:10:00 ILen:6 TLen:3 Toff:0 TAvgLen:6 Residual:0 Hrly:0 Sampl:6 RandStd:0 Th:0.5
Using BalGRUAdvPONI_addponi model
[Forecaster_addPONI] TargetLen:3 TeacherForcing:-1
[EncoderParams] channel_count:2 Shape:(540, 420)
[Discriminator] Downsample:3
[PoniModel_addponi W:0.1] Ckp:RF_71231_81231_mt-18_dt-81_lt-0_tlen-3_ilen-6_sampl-6_teaf--1_DisD-3_v-0 


In [7]:
model.eval()
with torch.no_grad():
    #cuda0 = torch.device('cuda:0')
    tmp_output = [] 
    tmp_target = [] 
    for batch in tqdm(loader):
        inp,target,mask = batch # 從dataloader取出來的資料還是tensor.cp
        inp = torch.where(inp < 1000, inp, 0.)
        inp= inp.cuda() # 將資料直接放到GPU上運算會比較快，在lightning中這行可不寫
        target = target.cuda()

        if model_kwargs['model_type'] in [18, 19, 20, 21]:
            addition = inp[:, :, 1:-1]
            inp = torch.cat([inp[:, :, 0:1], inp[:, :, -1:]], dim=2)
            output = model(inp, target, addition)
        elif model_kwargs['model_type'] in [6, 10] :
            output = model(inp)
        else:
            dummy = torch.zeros_like(target)
            output = model(inp, dummy)
        
        """
        if model_kwargs['model_type'] == 20:
            addition = inp[:, :, 1:-1]
            inp = torch.cat([inp[:, :, 0:1], inp[:, :, -1:]], dim=2)
            output = model(inp, target, addition)
        elif model_kwargs['model_type'] == 10:
            output = model(inp)
        else:
            output = model(inp, target)
        """
        
        output[output < 0] = 0
        output = output.cpu().numpy()
        target = target.cpu().numpy()
        tmp_output.append(output)
        tmp_target.append(target)
                

tmp_output = np.concatenate(tmp_output, axis=1)
tmp_target = np.concatenate(tmp_target, axis=0)
# convert tmp_output shape to [B, 3, H, W]
tmp_output = tmp_output.transpose([1,0,2,3])             

100%|██████████| 275/275 [03:45<00:00,  1.22it/s]


In [8]:
print(inp.shape)
k = inp
# k = torch.where(k < 1000, k, 0.)
print(torch.max(k))
print(dataset.initial_time(0))
print(tmp_output.shape)
# print(tmp_output[-5,...])
# print(np.max(tmp_output[-5,...]))
# print(dataset.initial_time(-5))

torch.Size([11, 6, 2, 540, 420])
tensor(13.2000, device='cuda:0')
2021-12-31 01:00:00
(8779, 3, 540, 420)


In [9]:
# pd = performance(tmp_output[:,0], tmp_target[:,0])
# pd.showCalResult()

## Save as pkl

In [10]:
import pickle
init_t = [dataset.initial_time(i) for i in range(len(dataset))]
# create dest_dir
if idx_lcl:
    dest_path = os.path.join('/work/xc3vancechen/data/model_output/repro_data', 
                            ModelType.name(model_kwargs['model_type'])+'_mt'+str(kwargs['mt'])+'_dt'+str(kwargs['dt'])+
                            '_lcl-1'+'_v-'+str(kwargs['v'])+'_hourly_2022.pkl')
else:
    dest_path = os.path.join('/work/xc3vancechen/data/model_output/repro_data', 
                            ModelType.name(model_kwargs['model_type'])+'_mt'+str(kwargs['mt'])+'_dt'+str(kwargs['dt'])+
                              '_v-'+str(kwargs['v'])+'_hourly_2022_mk2.pkl')

with open(dest_path, 'wb') as handle:
    pickle.dump([init_t, tmp_output, tmp_target], handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# numpy save
# np.savez_compressed("/".join(path) + '/evaluation/PONI_halfAtten_time_BF2', 
#                     model_output = tmp_output,
#                     target = tmp_target,
#                    )

## Save as nc file

In [11]:

# from netCDF4 import Dataset
# import numpy.ma as ma

# init_t = [dataset.initial_time(i) for i in range(len(dataset))]

# for tt in tqdm(range(len(init_t))):

#     # if init_t[tt].minute == 0:
#     if init_t[tt].day == 7:
#         # print(init_t)

#         fpath = os.path.join('/work/xc3vancechen/data/model_output'
#                               , "CAPN_nc_files", f"{init_t[tt].year}", f"{init_t[tt].year}{init_t[tt].month:02}"
#                               , f"{init_t[tt].year}{init_t[tt].month:02}{init_t[tt].day:02}")
#         fname = os.path.join(fpath, "deepQPF_"+f"{init_t[tt].year}{init_t[tt].month:02}{init_t[tt].day:02}_{init_t[tt].hour:02}{init_t[tt].minute:02}"+".nc")

#         if not os.path.exists(os.path.dirname(fname)):
#                 os.makedirs(os.path.dirname(fname), exist_ok=True) # os.path.dirname(fname) = fpath

#         latStart = 21.45; latEnd = 25.63; 
#         lonStart = 118.73; lonEnd = 123.05; 
#         lat = np.linspace(latStart,latEnd,tmp_output.shape[-2])
#         lon = np.linspace(lonStart,lonEnd,tmp_output.shape[-1])
#         time = np.arange(0,3)
        

#         f = Dataset(fname, 'w', format = 'NETCDF4')
#         f.createDimension('time', len(time))
#         f.createDimension('lat', len(lat))  
#         f.createDimension('lon', len(lon))
#         f.createVariable('output_rr', np.float32, ('time','lat', 'lon'))
#         f.createVariable('qperr', np.float32, ('time','lat', 'lon'))
#         f.createVariable('time', np.float32, ('time'))
#         f.createVariable('lat', np.float32, ('lat'))  
#         f.createVariable('lon', np.float32, ('lon'))
#         f.variables['time'][:] = np.array(time)
#         f.variables['lat'][:] = np.array(lat)
#         f.variables['lon'][:] = np.array(lon)
#         f.variables['output_rr'][:] = ma.masked_array(tmp_output[tt,:,:,:], mask=None)
#         f.variables['qperr'][:] = ma.masked_array(tmp_target[tt,:,:,:], mask=None)
#         f.close()
